# argilla

In [1]:
!pip install argilla

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.3/161.3 kB 3.4 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 372.7 kB/s eta 0:00:00 0:00:01
  Created wheel for standardwebhooks: filename=standardwebhooks-1.0.0-py3-none-any.whl size=3573 sha256=8cf75847437f502e18e07fa1cdfac858b2ee8d3e6388fda22f0770c4b1949780
  Stored in directory: /root/.cache/pip/wheels/cf/9b/f2/c580a7969ef91c332941343adf2fcc2c969b7fd3c9c8fa6add
Successfully built standardwebhooks


In [ ]:
import argilla as rg

HF_TOKEN = "xxx"

client = rg.Argilla(
    api_url="https://yiwenx-my-argilla.hf.space",
    api_key="Nvs8lnmsCXMq1b6O4com7yN-cpH5eKSFpYcNx1ghajQB_dLx0-A5IW9-vCpXNhq1qRfbzixF9b9OlvTk7zaahezC7Bnzz29PKRcL9YWTHBI",
    headers={"Authorization": f"Bearer {HF_TOKEN}"},
)

In [3]:
client.me

User(id=UUID('9bac15c3-1dfe-48cc-8787-618d2f538cad') inserted_at=datetime.datetime(2025, 9, 26, 8, 29, 11, 878434) updated_at=datetime.datetime(2025, 9, 26, 8, 29, 11, 878434) username='yiwenX' role=<Role.owner: 'owner'> first_name='yiwenX' last_name=None password=None)

In [4]:
from datasets import load_dataset

data = load_dataset("SetFit/ag_news", split="train")
data.features

train.jsonl:   0%|          | 0.00/33.8M [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

{'text': Value('string'),
 'label': Value('int64'),
 'label_text': Value('string')}

In [5]:
settings = rg.Settings(
    fields=[rg.TextField(name="text")],
    questions=[
        rg.LabelQuestion(
            name="label", title="Classify the text:", labels=data.unique("label_text")
        ),
        rg.SpanQuestion(
            name="entities",
            title="Highlight all the entities in the text:",
            labels=["PERSON", "ORG", "LOC", "EVENT"],
            field="text",
        ),
    ],
)

In [6]:
dataset = rg.Dataset(name="ag_news", settings=settings)

dataset.create()

/root/miniconda3/lib/python3.12/site-packages/argilla/datasets/_resource.py:264: UserWarning: Workspace not provided. Using default workspace: argilla id: 21de30ac-e415-477f-b6fd-af2c1b52aca0
  warnings.warn(f"Workspace not provided. Using default workspace: {workspace.name} id: {workspace.id}")


Dataset(id=UUID('1dae4b66-dbaf-4077-8254-a9fb580cf9e1') inserted_at=datetime.datetime(2025, 9, 26, 8, 32, 53, 388124) updated_at=datetime.datetime(2025, 9, 26, 8, 32, 54, 775026) name='ag_news' status='ready' guidelines=None allow_extra_metadata=False distribution=OverlapTaskDistributionModel(strategy='overlap', min_submitted=1) workspace_id=UUID('21de30ac-e415-477f-b6fd-af2c1b52aca0') last_activity_at=datetime.datetime(2025, 9, 26, 8, 32, 54, 775026))

In [7]:
dataset.records.log(data, mapping={"label_text": "label"})

/root/miniconda3/lib/python3.12/site-packages/argilla/records/_io/_datasets.py:265: UserWarning: Record id column not found in Hugging Face dataset. Using row index and split for record ids.
  warnings.warn(


Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

/root/miniconda3/lib/python3.12/site-packages/argilla/records/_mapping/_mapper.py:89: UserWarning: Keys ['label_text'] in data are not present in the mapping and will be ignored.
  warnings.warn(f"Keys {unknown_keys} in data are not present in the mapping and will be ignored.")
Sending records...: 469batch [21:51,  2.80s/batch]                      


DatasetRecords(Dataset(id=UUID('1dae4b66-dbaf-4077-8254-a9fb580cf9e1') inserted_at=datetime.datetime(2025, 9, 26, 8, 32, 53, 388124) updated_at=datetime.datetime(2025, 9, 26, 8, 32, 54, 775026) name='ag_news' status='ready' guidelines=None allow_extra_metadata=False distribution=OverlapTaskDistributionModel(strategy='overlap', min_submitted=1) workspace_id=UUID('21de30ac-e415-477f-b6fd-af2c1b52aca0') last_activity_at=datetime.datetime(2025, 9, 26, 8, 32, 54, 775026)))

In [8]:
status_filter = rg.Query(filter=rg.Filter([("status", "==", "completed")]))

filtered_records = dataset.records(status_filter)

In [10]:
filtered_records.to_datasets().push_to_hub("ag_news_annotated")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format: 0ba [00:00, ?ba/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

                                        : 100%|##########|   324B /   324B            

CommitInfo(commit_url='https://huggingface.co/datasets/yiwenX/ag_news_annotated/commit/a4ec53d59c1b4c999abb6aedc42aa9a5d1be5d01', commit_message='Upload dataset', commit_description='', oid='a4ec53d59c1b4c999abb6aedc42aa9a5d1be5d01', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/yiwenX/ag_news_annotated', endpoint='https://huggingface.co', repo_type='dataset', repo_id='yiwenX/ag_news_annotated'), pr_revision=None, pr_num=None)